In [68]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
import xgboost as xgb
# from mlxtend.classifier import StackingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit

import warnings
warnings.filterwarnings('ignore')

In [2]:
cd royal/Downloads

/home/royal/Downloads


In [95]:
train=pd.read_csv('train_u6lujuX_CVtuZ9i.csv')
test=pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')
sample=pd.read_csv('Sample_Submission_ZAuTl8O_FK3zQHh.csv')
ID=test['Loan_ID']

In [96]:
missing_df = train.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
missing_df

,column_name,missing_count
2,Married,3
1,Gender,13
9,Loan_Amount_Term,14
3,Dependents,15
8,LoanAmount,22
5,Self_Employed,32
10,Credit_History,50


In [97]:
missing_df = test.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
missing_df

,column_name,missing_count
8,LoanAmount,5
9,Loan_Amount_Term,6
3,Dependents,10
1,Gender,11
5,Self_Employed,23
10,Credit_History,29


In [98]:
train.Gender=train.Gender.fillna(train.Gender.mode()[0])
train.Married=train.Married.fillna(train.Married.mode()[0])
train.Loan_Amount_Term=train.Loan_Amount_Term.fillna(train.Loan_Amount_Term.mode()[0])
train.Dependents=train.Dependents.fillna(train.Dependents.mode()[0])
train.LoanAmount=train.LoanAmount.fillna(train.LoanAmount.mean())
train.Self_Employed=train.Self_Employed.fillna(train.Self_Employed.mode()[0])
train.Credit_History=train.Credit_History.fillna(train.Credit_History.mode()[0])






In [99]:

test.Gender=test.Gender.fillna(test.Gender.mode()[0])
test.Loan_Amount_Term=test.Loan_Amount_Term.fillna(test.Loan_Amount_Term.mode()[0])
test.Dependents=test.Dependents.fillna(test.Dependents.mode()[0])
test.LoanAmount=test.LoanAmount.fillna(test.LoanAmount.mean())
test.Self_Employed=test.Self_Employed.fillna(test.Self_Employed.mode()[0])
test.Credit_History=test.Credit_History.fillna(test.Credit_History.mode()[0])

In [100]:
from sklearn import preprocessing, model_selection, metrics

for f in test.columns :
        if (train[f].dtype=='object' and f!='Loan_ID'):
           
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[f].values))
            train[f] = lbl.transform(list(train[f].values))
            test[f] = lbl.transform(list(test[f].values))

In [101]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,146.412162,360.0,1.0,2,Y
1,LP001003,1,1,1,0,0,4583,1508.0,128.000000,360.0,1.0,0,N
2,LP001005,1,1,0,0,1,3000,0.0,66.000000,360.0,1.0,2,Y
3,LP001006,1,1,0,1,0,2583,2358.0,120.000000,360.0,1.0,2,Y
4,LP001008,1,0,0,0,0,6000,0.0,141.000000,360.0,1.0,2,Y


In [102]:
train.Gender=train.Gender.astype('category')
train.Married=train.Married.astype('category')
train.Dependents=train.Dependents.astype('category')
train.Education=train.Education.astype('category')
train.Self_Employed=train.Self_Employed.astype('category')
train.Credit_History=train.Credit_History.astype('category')
train.Property_Area=train.Property_Area.astype('category')
train.Loan_Status=train.Loan_Status.astype('category')


In [103]:
test.Gender=test.Gender.astype('category')
test.Married=test.Married.astype('category')
test.Dependents=test.Dependents.astype('category')
test.Education=test.Education.astype('category')
test.Self_Employed=test.Self_Employed.astype('category')
test.Credit_History=test.Credit_History.astype('category')
test.Property_Area=test.Property_Area.astype('category')



In [104]:
train['ApplicantIncome']=np.log1p(train['ApplicantIncome'])
train['CoapplicantIncome']=np.log1p(train['CoapplicantIncome'])
train['LoanAmount']=np.log1p(train['LoanAmount'])

test['ApplicantIncome']=np.log1p(test['ApplicantIncome'])
test['CoapplicantIncome']=np.log1p(test['CoapplicantIncome'])
test['LoanAmount']=np.log1p(test['LoanAmount'])


In [105]:
# NEw features



# train['CbyA']=train['CoapplicantIncome']/train['ApplicantIncome']
train['CtoA']=train['CoapplicantIncome']*train['ApplicantIncome']

# train['LbyLT']=train['LoanAmount']/train['Loan_Amount_Term']
train['AtoLT']=train['ApplicantIncome']*train['Loan_Amount_Term']
train['CtoLT']=train['CoapplicantIncome']*train['Loan_Amount_Term']


# test['CbyA']=train['CoapplicantIncome']/test['ApplicantIncome']
# test['LbyLT']=test['LoanAmount']/test['Loan_Amount_Term']
test['AtoLT']=test['ApplicantIncome']*test['Loan_Amount_Term']
test['CtoA']=test['CoapplicantIncome']*test['ApplicantIncome']
test['CtoLT']=test['CoapplicantIncome']*test['Loan_Amount_Term']

train['Very_Short_Term'] = train['Loan_Amount_Term'].map(lambda t: 1 if t<=60 else 0)
train['Short_Term'] = train['Loan_Amount_Term'].map(lambda t: 1 if t>60 and t<180 else 0)
train['Long_Term'] = train['Loan_Amount_Term'].map(lambda t: 1 if t>=180 and t<=300  else 0)
train['Very_Long_Term'] = train['Loan_Amount_Term'].map(lambda t: 1 if t>300 else 0)
train.drop('Loan_Amount_Term', axis=1, inplace=True)


test['Very_Short_Term'] = test['Loan_Amount_Term'].map(lambda t: 1 if t<=60 else 0)
test['Short_Term'] = test['Loan_Amount_Term'].map(lambda t: 1 if t>60 and t<180 else 0)
test['Long_Term'] = test['Loan_Amount_Term'].map(lambda t: 1 if t>=180 and t<=300  else 0)
test['Very_Long_Term'] = test['Loan_Amount_Term'].map(lambda t: 1 if t>300 else 0)
test.drop('Loan_Amount_Term', axis=1, inplace=True)

train['Credit_History_Bad'] = train['Credit_History'].map(lambda c: 1 if c==0 else 0)
train['Credit_History_Good'] = train['Credit_History'].map(lambda c: 1 if c==1 else 0)
train.drop('Credit_History', axis=1, inplace=True)




test['Credit_History_Bad'] = test['Credit_History'].map(lambda c: 1 if c==0 else 0)
test['Credit_History_Good'] = test['Credit_History'].map(lambda c: 1 if c==1 else 0)
test.drop('Credit_History', axis=1, inplace=True)



In [106]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesClassifier

In [107]:
train['Loan_Status']=train['Loan_Status'].apply(lambda x :1 if x=='Y' else 0 )

In [108]:
X=train.drop(['Loan_Status','Loan_ID'],axis=1)
Y=train['Loan_Status']
test=test.drop(['Loan_ID'],axis=1)

In [109]:
from sklearn.model_selection import train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [110]:
X.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Property_Area,CtoA,AtoLT,CtoLT,Very_Short_Term,Short_Term,Long_Term,Very_Long_Term,Credit_History_Bad,Credit_History_Good
0,1,0,0,0,0,8.674197,0.000000,4.993232,2,0.000000,3122.710898,0.000000,0,0,0,1,0,1
1,1,1,1,0,0,8.430327,7.319202,4.859812,0,61.703272,3034.917813,2634.912885,0,0,0,1,0,1
2,1,1,0,0,1,8.006701,0.000000,4.204693,2,0.000000,2882.412304,0.000000,0,0,0,1,0,1
3,1,1,0,1,0,7.857094,7.765993,4.795791,2,61.018137,2828.553791,2795.757509,0,0,0,1,0,1
4,1,0,0,0,0,8.699681,0.000000,4.955827,2,0.000000,3131.885304,0.000000,0,0,0,1,0,1


In [113]:
train.dtypes

Loan_ID                  object
Gender                 category
Married                category
Dependents             category
Education              category
Self_Employed          category
ApplicantIncome         float64
CoapplicantIncome       float64
LoanAmount              float64
Property_Area          category
Loan_Status            category
CtoA                    float64
AtoLT                   float64
CtoLT                   float64
Very_Short_Term           int64
Short_Term                int64
Long_Term                 int64
Very_Long_Term            int64
Credit_History_Bad     category
Credit_History_Good    category
dtype: object

In [115]:
svcmod = SVC(C = 15, kernel = 'rbf', gamma = 0.2, probability= True)
svcmod.scores = cross_val_score(svcmod, dev_X, dev_y, cv=5)

knnmod = KNeighborsClassifier(n_neighbors= 10)
knnmod.scores = cross_val_score(knnmod, dev_X, dev_y, cv=5)

rfmod = RandomForestClassifier(n_estimators=100, max_features=10, max_depth=5)
rfmod.scores = cross_val_score(rfmod, dev_X, dev_y, cv=5)

# xgbmod = xgb.XGBClassifier(learning_rate= 0.1, gamma = 0.8, max_depth= 3, min_child_weight=7, 
#                            objective= 'binardev_y:logistic', subsample=0.8, colsample_bytree=0.8, 
#                            scale_pos_weight=1, n_estimators = 200, nthread = -1)
# xgbmod.scores = cross_val_score(xgbmod, dev_X, dev_y, cv=5)

etmod = ExtraTreesClassifier(n_estimators=300, max_features=6, max_depth=6)
etmod.scores = cross_val_score(etmod, dev_X, dev_y, cv=5)

admod = AdaBoostClassifier(n_estimators= 100, learning_rate= 0.1)
admod.scores = cross_val_score(admod, dev_X, dev_y, cv=5)

lr = LogisticRegression(C = 0.8)
lr.scores = cross_val_score(lr, dev_X, dev_y, cv=5)

for clf, label in zip([svcmod, knnmod, rfmod, etmod, admod, lr], 
                      ['SVC', 
                       'KNN', 
                       'Random Forest',
                       'ExtraTrees',
                       'AdaBoost',
                       'Logistic Regression']):
    print("%s Accuracy: %0.3f" % (label,clf.scores.mean()))

SVC Accuracy: 0.686
KNN Accuracy: 0.686
Random Forest Accuracy: 0.796
ExtraTrees Accuracy: 0.813
AdaBoost Accuracy: 0.806
Logistic Regression Accuracy: 0.815


In [117]:
votemod = VotingClassifier(estimators=[('et', etmod),('lr',lr),
                                       ('rf', rfmod), ('ad', admod)], 
                           weights = [2,1,2,1,],
                           voting='soft')
votemod.scores = cross_val_score(votemod, dev_X, dev_y, cv=5)
print(votemod.scores.mean())

0.8145456245603521


In [66]:
# parameters = {'bootstrap': True,
#               'min_samples_leaf': 5,
#               'n_estimators':4000,
#               'n_jobs':-1,
#               'min_samples_split': 20,
#               'max_features': 'auto',
#               'max_depth': 16}

m= ExtraTreesClassifier(n_estimators=400)

m.fit(dev_X,dev_y)
m.score(val_X,val_y)

0.7804878048780488

In [57]:
z = m.feature_importances_
df = pd.DataFrame()
df["values"] = z*100
df["column"] = list(dev_X.columns.values)
df

,values,column
0,2.306942,Gender
1,2.178379,Married
2,5.130250,Dependents
3,2.320661,Education
4,1.829683,Self_Employed
5,12.086731,ApplicantIncome
6,4.583646,CoapplicantIncome
7,11.118590,LoanAmount
8,5.344648,Property_Area
9,4.082424,CtoA


In [118]:
votemod.fit(X,Y)
ypred=votemod.predict(test)

In [119]:
sample['Loan_Status']=ypred

In [120]:
sample['Loan_ID']=ID

In [121]:
sample['Loan_Status'].value_counts()

1    288
0     79
Name: Loan_Status, dtype: int64

In [125]:
sample.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y


In [126]:
sample.to_csv('submis.csv',index=False)

In [124]:
sample['Loan_Status']=sample['Loan_Status'].apply(lambda x:'Y' if x==1 else 'N')